In [18]:
import time
import hmac
import requests
import pandas as pd
from datetime import datetime
import urllib.parse

In [172]:
class FTX_Client_Hicham:
    api_endpoint_base = "https://ftx.com/api/"

    def __init__(self,api_key,api_secret,subbacount_name):
        self._session = requests.Session()
        self._api_key = api_key
        self._api_secret = api_secret
        self._subbacount_name = subbacount_name

    def get_open_orders(self):
        url = self.api_endpoint_base + "orders"
        request = requests.Request('GET', url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        if len(df_data)!=0:
            df_data.drop(['clientId','reduceOnly','liquidation','postOnly','ioc',"future","createdAt","avgFillPrice","remainingSize","filledSize"],axis=1,inplace=True) 
            df_data.set_index("id",inplace=True)    
        return df_data

    def get_balances(self):
        url = self.api_endpoint_base + "wallet/balances"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        return df_data[df_data["total"]>0]

    def get_order_history(self):
        url = self.api_endpoint_base + "orders/history"
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)     
        return df_data   

    def get_last_funding_rates(self,market):
        url = self.api_endpoint_base + "funding_rates?future=" + market
        request = requests.Request("GET",url)
        data = self.send_request(request)
        df_data = pd.DataFrame(data)
        df_data.set_index("time",inplace=True)
        return df_data[:5]['rate'].to_numpy()

    def send_order(self,market,side,price,type,size):
        url = self.api_endpoint_base + "orders"
        params = {"market":market,"side":side,"price":price,"type":type,"size":size}
        request = requests.Request("POST",url,json=params)
        data = self.send_request(request)
        return data

    def process_response(self,response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']

    def send_request(self,request):
        ts = int(time.time() * 1000)
        prepared = request.prepare()

        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()

        if prepared.body:
            signature_payload += prepared.body

        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        prepared.headers[f'FTX-KEY'] = self._api_key
        prepared.headers[f'FTX-SIGN'] = signature
        prepared.headers[f'FTX-TS'] = str(ts)
        prepared.headers[f'FTX-SUBACCOUNT'] = urllib.parse.quote(self._subbacount_name)

        response = self._session.send(prepared)
        return self.process_response(response) 


    def get_live_quote(self,base_currency:str,quote_currency:str):
        """Permet d'avoir les quotes pour les spots et les futures perpetuels, par ex: get_quote("BTC","USD") ou get_quote("BTC","PERP")"""
       
        if quote_currency.casefold == "PERP".casefold:
            api_endpoint = self.api_endpoint_base + "markets/" + base_currency + "-" + quote_currency
        else : 
            api_endpoint = self.api_endpoint_base + "markets/" + base_currency + "/" + quote_currency
            
        req = requests.get(api_endpoint).json()
        return pd.DataFrame(req)['result']

    def get_historical_data(self,base_currency:str,quote_currency:str,resolution:int):
        """Permet d'avoir les données historiques sur une paire base/quote ou sur un futur perpétuel, la résolution est à indiquer en secondes"""

        if quote_currency.casefold == "PERP".casefold:
            api_endpoint = self.api_endpoint_base + "markets/" + base_currency + "-" + quote_currency
        else : 
            api_endpoint = self.api_endpoint_base + "markets/" + base_currency + "/" + quote_currency   

        api_endpoint += "/candles?resolution=" + str(resolution)
        req = requests.get(api_endpoint).json()
        df = pd.DataFrame(req['result'])
        df['date'] = pd.to_datetime(df["time"]/1000,unit='s',origin='unix')
        df.drop(['startTime','time'],axis=1,inplace=True)  
        df.set_index('date',inplace=True)
        return df 

In [121]:
client = FTX_Client_Hicham(api_key,api_secret,'TestApi')
orders = client.get_open_orders()
print(orders)

                market   type side   price  size status
id                                                     
144715975589  USDT/USD  limit  buy  0.9999   1.0   open
144707575878  USDT/USD  limit  buy  0.5000   1.0   open


In [120]:
client = FTX_Client_Hicham(api_key,api_secret,'TestApi')
client.send_order("USDT/USD","buy","0.9999","limit","1")

{'id': 144715975589,
 'clientId': None,
 'market': 'USDT/USD',
 'type': 'limit',
 'side': 'buy',
 'price': 0.9999,
 'size': 1.0,
 'status': 'new',
 'filledSize': 0.0,
 'remainingSize': 1.0,
 'reduceOnly': False,
 'liquidation': None,
 'avgFillPrice': None,
 'postOnly': False,
 'ioc': False,
 'createdAt': '2022-05-10T15:12:26.927599+00:00',
 'future': None}

In [123]:
client = FTX_Client_Hicham(api_key,api_secret,'TestApi')
client.get_order_history()

,id,clientId,market,type,side,price,size,status,filledSize,remainingSize,reduceOnly,liquidation,avgFillPrice,postOnly,ioc,createdAt,future
0,144715975589,None,USDT/USD,limit,buy,0.9999,1.0,closed,0.0,0.0,False,False,None,False,False,2022-05-10T15:12:26.927599+00:00,None
1,144707575878,None,USDT/USD,limit,buy,0.5000,1.0,closed,0.0,0.0,False,False,None,False,False,2022-05-10T14:50:14.191306+00:00,None
2,144693805337,None,USDT/USD,limit,buy,0.5000,1.0,closed,0.0,0.0,False,False,None,False,False,2022-05-10T14:11:03.841764+00:00,None
3,144688160286,None,SRM/USD,limit,buy,1.0000,1.0,closed,0.0,0.0,False,False,None,False,False,2022-05-10T13:55:05.574823+00:00,None
4,144682899474,None,USDT/USD,limit,buy,0.5000,1.0,closed,0.0,0.0,False,False,None,False,False,2022-05-10T13:40:07.677076+00:00,None


In [164]:
client = FTX_Client_Hicham(api_key,api_secret,"TestApi")
fdg_rates = client.get_last_funding_rates("BTC-PERP")
fdg_rates

array([1.0e-06, 2.0e-06, 1.3e-05, 1.4e-05, 9.0e-06])

In [173]:
client = FTX_Client_Hicham(api_key,api_secret,"TestApi")
fdg_rates = client.get_balances()
fdg_rates

,coin,total,free,availableForWithdrawal,availableWithoutBorrow,usdValue,spotBorrow
5,USD,3.0,3.0,3.0,3.0,3.0,0.0
